# Operation Private Ryan
### Patch the prediction, to gaurantee at least a  single child

### Using data leak

All picture must has at least 1 class of prediction

Patch the prediction with safenet

In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [36]:
import cv2
import os
from tqdm import tqdm
from pathlib import Path

Configs

In [37]:
MIN_SIZE = 1000
HOME = Path(os.environ["HOME"])
INPUT_DIR = Path(HOME/"ucsi")
SUBS = INPUT_DIR/"subs"

Check sub files

In [38]:
!ls -l {SUBS}

total 338276
-rw-rw-r-- 1 salvor salvor 21579597 Nov 16 15:10 1110_154142_submission.csv
-rw-rw-r-- 1 salvor salvor 21251497 Nov 16 15:10 1114_090732_submission.csv
-rw-rw-r-- 1 salvor salvor 21085658 Nov 16 15:10 b5_fold_optimize_trian0.5_1114_150440_submission.csv
-rw-rw-r-- 1 salvor salvor 22902693 Nov 16 15:10 b5_pla_0.72_1573548650_submission.csv
-rw-rw-r-- 1 salvor salvor 23107546 Nov 16 15:10 b5_r2_1573525424_submission.csv
-rw-rw-r-- 1 salvor salvor 23211737 Nov 16 15:11 b6_fold_1114_121001_submission.csv
-rw-rw-r-- 1 salvor salvor 22315669 Nov 16 15:11 b6_ranger_submission.csv
-rw-rw-r-- 1 salvor salvor 20280347 Nov 16 15:11 b6_ver2_submission.csv
-rw-rw-r-- 1 salvor salvor 21339234 Nov 16 15:11 convex_1110_154142_submission.csv
-rw-rw-r-- 1 salvor salvor 20998002 Nov 16 15:11 convex_1111_161204_submission.csv
-rw-rw-r-- 1 salvor salvor 20853638 Nov 16 15:11 convex_b5_fold_optimize_trian0.5_1114_150440_submission.csv
-rw-rw-r-- 1 salvor salvor 21170814 Nov 16 15:11 convex_b6_f

Set submission csv pool

In [39]:
CSV = SUBS/"emp_convex_1110_154142_submission.csv"
df = pd.read_csv(CSV)

In [40]:
df.head()

,Image_Label,EncodedPixels
0,63bb93e.jpg_Fish,NaN
1,63bb93e.jpg_Flower,NaN
2,63bb93e.jpg_Gravel,3355 14 3702 23 4049 32 4397 40 4746 48 5094 5...
3,63bb93e.jpg_Sugar,4094 10 4442 15 4790 21 5139 26 5487 32 5836 3...
4,8926664.jpg_Fish,NaN


In [41]:
def checkWorthy(df):
    """
    Check if the prediction is worthy
    """
    df["img"] = df.Image_Label.apply(lambda x: x.split("_")[0])
    df["cls"] = df.Image_Label.apply(lambda x: x.split("_")[1])
    df["mark"] = (~df.EncodedPixels.isnull())*1
    img_count = df.groupby("img").sum()[["mark"]].reset_index()
    missing = list(img_count[img_count.mark ==0]["img"])
    df["worthy"] = ~df.img.isin(missing)
    df = df.drop(["img","cls","mark"],axis=1)
    print(df.worthy.value_counts())
    return df,missing

In [42]:
df,missing = checkWorthy(df)

True     14608
False      184
Name: worthy, dtype: int64


In [43]:
print(len(missing), "pics are missing class")
for m in missing: print(m, end = "\t")

46 pics are missing class
00d2a33.jpg	09a8f3a.jpg	0c3cc90.jpg	1a8aae8.jpg	2178dcf.jpg	230437f.jpg	2d1a55a.jpg	2ff10e7.jpg	41648e6.jpg	4375f9e.jpg	45766df.jpg	4c87181.jpg	4df033a.jpg	57dff72.jpg	5e75d4a.jpg	61e3406.jpg	642383d.jpg	6aac900.jpg	750e940.jpg	7647f9e.jpg	7927611.jpg	7b669a8.jpg	8d7d0c5.jpg	98a7545.jpg	9931011.jpg	9d112ac.jpg	9dde405.jpg	a227703.jpg	a252165.jpg	a366cab.jpg	a6ef1d3.jpg	a88da3a.jpg	acee09d.jpg	bbb52ed.jpg	bd7a8eb.jpg	bfcd369.jpg	c65b88c.jpg	c89044d.jpg	d0da6f1.jpg	d6c90c7.jpg	d83724a.jpg	d966353.jpg	dd9b63c.jpg	e053e5e.jpg	e57e222.jpg	edda79c.jpg	

In [44]:
# This has to be an odd number
sub_fnames = list([
    "resnext_miracle_raw.csv",# resnext 101 1st miracle 0.6596
    "emp_convex_384x567_1117_211214_submission.csv",
#     ("1110_154142_submission.csv",1/7), # enesemble 
#     ("b5_pla_0.72_1573548650_submission.csv",0.33333333)
#     ("b5_r2_1573525424_submission.csv",0.3333333)finding
#     ("dpn_131_0.73_1573574797_submission.csv",0.333333) #0.6443
    "b5_fold_optimize_trian0.5_1114_150440_submission.csv", # 0.6587
#     ("b6_ver2_submission.csv",1/7), #0.6527
    "b6_ranger_submission.csv", #0.6556
    "1114_090732_submission.csv", # resnext 101 5 folds 0.6569
    "b6_fold_1114_121001_submission.csv", # b6 6 folds  0.6558
    "convex_b5_fold_optimize_trian0.5_1114_150440_submission.csv", # convex b5 5 folds
    "convex_b6_fold_train0.5_optima_1116_080218_submission.csv", # convex b6 5 bolds
])
sub_paths = list(SUBS/p for p in sub_fnames)

In [45]:
def bringOrder(df):
    return df.sort_values(by="Image_Label", ascending=True).reset_index().drop("index",axis=1)

In [46]:
from itertools import chain

In [47]:
miss_labels = []
for m in missing:
    miss_labels = chain(miss_labels,list("%s_%s"%(m,c) for c in ["Fish","Flower","Gravel","Sugar"]))
miss_list = list(miss_labels)

In [48]:
len(miss_list)

184

In [49]:
sub_dfs = list(checkWorthy(bringOrder(pd.read_csv(p)))[0] for p in sub_paths) 
sub_dfs = list(df[df.Image_Label.isin(miss_list)] for df in sub_dfs)
sub_dfs = list(df[df.worthy].fillna("") for df in sub_dfs)
sample_df = sub_dfs[0]

True     14532
False      260
Name: worthy, dtype: int64
True     14784
False        8
Name: worthy, dtype: int64
True     14488
False      304
Name: worthy, dtype: int64
True     14584
False      208
Name: worthy, dtype: int64
True     14624
False      168
Name: worthy, dtype: int64
True     14660
False      132
Name: worthy, dtype: int64
True     14488
False      304
Name: worthy, dtype: int64
True     14512
False      280
Name: worthy, dtype: int64


### Statistic on patch helpers

In [50]:
work_df = pd.DataFrame({"Image_Label":miss_list})
i = 0
for help_df in sub_dfs:
    lbl_map = dict(zip(help_df.Image_Label,help_df.worthy))
    def getHelp(x):
        try:
            return lbl_map[x]
        except:
            return False
    work_df["h_%s"%(i)] = work_df.Image_Label.apply(getHelp)
    i+=1
cols = list("h_%s"%(h) for h in range(i))
work_df["total_worthy"] = work_df.apply(lambda x: sum(list(x[c]for c in cols)), axis=1)

In [51]:
work_df

,Image_Label,h_0,h_1,h_2,h_3,h_4,h_5,h_6,h_7,total_worthy
0,00d2a33.jpg_Fish,False,True,False,False,False,False,False,False,1
1,00d2a33.jpg_Flower,False,True,False,False,False,False,False,False,1
2,00d2a33.jpg_Gravel,False,True,False,False,False,False,False,False,1
3,00d2a33.jpg_Sugar,False,True,False,False,False,False,False,False,1
4,09a8f3a.jpg_Fish,False,True,False,False,True,True,False,False,3
5,09a8f3a.jpg_Flower,False,True,False,False,True,True,False,False,3
6,09a8f3a.jpg_Gravel,False,True,False,False,True,True,False,False,3
7,09a8f3a.jpg_Sugar,False,True,False,False,True,True,False,False,3
8,0c3cc90.jpg_Fish,False,True,False,False,False,True,False,False,2
9,0c3cc90.jpg_Flower,False,True,False,False,False,True,False,False,2


In [52]:
print(pd.DataFrame(work_df.total_worthy.value_counts()))

   total_worthy
2            40
4            32
1            32
8            20
3            20
7            16
6             8
5             8
0             8


In [53]:
print("How many rows of problem we can solve")
print(pd.DataFrame(work_df.total_worthy.apply(lambda x:x>0).value_counts()))

How many rows of problem we can solve
       total_worthy
True            176
False             8


### Helper Functions

In [54]:

def rle_decode(mask_rle: str = '', shape = (350,525 )):
    '''
    Decode rle encoded mask.
    
    :param mask_rle: run-length as string formatted (start length)
    :param shape: (height, width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    
    return img.reshape(shape, order='F')

def post_process(probability, threshold=0.5, min_size = MIN_SIZE):
    """
    Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored
    """
    # don't remember where I saw it
    mask = cv2.threshold(np.float32(probability), threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((350, 525), np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

def mask2rle(img):
    '''
    Convert mask to rle.
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [55]:
# def draw_convex_hull(mask, mode='convex'):
# #     mask = np.expand_dims(mask,axis=0)
#     mask = mask.astype(np.uint8)
#     img = np.zeros(mask.shape)
#     contours, hier = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
#     for c in contours:
#         if mode=='rect': # simple rectangle
#             x, y, w, h = cv2.boundingRect(c)
#             cv2.rectangle(img, (x, y), (x+w, y+h), (255, 255, 255), -1)
#         elif mode=='convex': # minimum convex hull
#             hull = cv2.convexHull(c)
#             cv2.drawContours(img, [hull], 0, (255, 255, 255),0)
#         elif mode=='approx':
#             epsilon = 0.02*cv2.arcLength(c,True)
#             approx = cv2.approxPolyDP(c,epsilon,True)
#             cv2.drawContours(img, [approx], 0, (255, 255, 255),-1)
#         else: # minimum area rectangle
#             rect = cv2.minAreaRect(c)
#             box = cv2.boxPoints(rect)
#             box = np.int0(box)
#             cv2.drawContours(img, [box], 0, (255, 255, 255),-1)
#     return img/255.

# def draw_masks(img2,img_mask_list):
# #     img2 = np.expand_dims(img2,axis=0)
# #     img_mask_list =list(np.expand_dims(a,axis=0) for a in img_mask_list)
#     img = img2.copy()
#     color_mask = np.zeros(img2.shape)
#     temp_mask = np.ones([img2.shape[0],img2.shape[1]])*127./255.
#     temp_mask[img_mask_list[0] == 0] = 0
#     color_mask[:,:] = temp_mask
#     img += color_mask
        
#     return np.clip(img,0.,1.)

In [56]:
i = 0
patch = dict()
for r in work_df.iterrows():
    i+=1
#     if i == 21: break
    row = r[1]
    label = row[0]
    total_worthy = row[-1]
    row_map = row[1:-1]
    dfs = list(sub_dfs[i] for i in range(len(row_map))if row_map[i])
#     print(label)
    patch_cluster = list(list(df_[df_.Image_Label==label]["EncodedPixels"])[0] for df_ in dfs)
    if len(patch_cluster)==1:
        print("[%s] has only 1 solution, no ensemble"%(label))
        if patch_cluster[0]!='':
            patch[label] = patch_cluster[0]
        continue
    if len(patch_cluster)==0:
        print("[%s] has no solution"%(label))
        continue
    pred_list = list(rle_decode(p) for p in patch_cluster)
    stacked = np.mean(np.stack(pred_list,axis=0),axis=0)
    stacked = post_process(stacked, threshold=0.2,min_size=1000)[0]
#     stacked = draw_masks(stacked,list([draw_convex_hull(stacked)]))
#     stacked = post_process(stacked)[0]
    newRLE = mask2rle(stacked)
    if newRLE!='':
        patch[label] = newRLE
print("total patch",len(patch))

[00d2a33.jpg_Fish] has only 1 solution, no ensemble
[00d2a33.jpg_Flower] has only 1 solution, no ensemble
[00d2a33.jpg_Gravel] has only 1 solution, no ensemble
[00d2a33.jpg_Sugar] has only 1 solution, no ensemble
[2d1a55a.jpg_Fish] has only 1 solution, no ensemble
[2d1a55a.jpg_Flower] has only 1 solution, no ensemble
[2d1a55a.jpg_Gravel] has only 1 solution, no ensemble
[2d1a55a.jpg_Sugar] has only 1 solution, no ensemble
[2ff10e7.jpg_Fish] has no solution
[2ff10e7.jpg_Flower] has no solution
[2ff10e7.jpg_Gravel] has no solution
[2ff10e7.jpg_Sugar] has no solution
[5e75d4a.jpg_Fish] has no solution
[5e75d4a.jpg_Flower] has no solution
[5e75d4a.jpg_Gravel] has no solution
[5e75d4a.jpg_Sugar] has no solution
[750e940.jpg_Fish] has only 1 solution, no ensemble
[750e940.jpg_Flower] has only 1 solution, no ensemble
[750e940.jpg_Gravel] has only 1 solution, no ensemble
[750e940.jpg_Sugar] has only 1 solution, no ensemble
[7927611.jpg_Fish] has only 1 solution, no ensemble
[7927611.jpg_Flower

In [57]:
df = df.drop("worthy",axis=1)

In [58]:
for img_,rle_ in patch.items():
    df.loc[df.Image_Label==img_,"EncodedPixels"]=rle_

df2,_ = checkWorthy(df)

True     14784
False        8
Name: worthy, dtype: int64


### Saving to submission CSV

In [59]:
df.to_csv('ryan_%s'%(CSV.name), columns=['Image_Label', 'EncodedPixels'], index=False)